# Test: Bestimmung Precision / Recall mit SVM
Um die Klassifikation, und damit auch von dem Aufbau unserer Matrix, zu testen, bestimmen wir Precision und Recall mit einer SVM.

* Precision: Wieviele der zugeordneten Samples sind tatsächlich relevant? (true pos.) / (true pos. + false pos.)
* Recall: Wieviele der tatsächlich relevanten Samples wurden gefunden / zugeordnet? (true pos.) / (true pos. + false neg.)
 * Tiefe Precision / hoher Recall: Es werden viele Samples zugeordnet, von denen jedoch viele falsch sind
 * Hohe Precision / tiefer Recall: Es werden nur wenige Samples zugeordnet, von denen jedoch die meisten stimmen.
<img src="img/Precisionrecall.png" alt="Drawing" style="width: 200px"/>

## Optimierung
Mit diesem Test versuchen wir die Konfiguration der SVM möglichst gut auf unsere Datenstruktur auszurichten.
### Kernel
Der Kernel definiert, mit welcher Art Funktion die Datenpunkte im 2D-Raum voneinander getrennt werden. Standardmässig wird dazu eine lineare Funktion (linear kernel) verwendet. Wenn die Punkte im training set nicht durch eine Gerade trennbar sind, versucht man sie in einer höheren Dimension voneinander zu trennen. Die erreicht man durch andere kernel-Funktionen (z.B. Polynomial oder RBF).
Bei vielen Features, wie es bei unseren Daten der Fall ist, bringt eine höhere Dimension keine Vorteile, ist aber sehr teuer. Entsprechend verwenden wir einen linearen Kernel.

#### Parameter
Der lineare Kernel verlangt ausschliesslich einen Penalty Parameter C. Beim einem hohen C versucht der Classifier die Ebene so zu legen, dass möglichst wenige Datenpunkte aus dem training set falsch klassifiziert sind. Dafür nimmt er ein kleineres Margin zwischen Ebene und Punkte in Kauf. Bei einem tiefen Wert für C sucht der Classifier nach einem möglichst hohen margin, nimmt aber falsche Klassifizierungen im trianing set in Kauf.


# Das folgende Skript ist für einen average test mit einer einzelnen Matrix

In [5]:
import matplotlib.pyplot as plt
import numpy as np

from imports.matrix_helper import MatrixHelper
from imports.prediction_helper import PredictionHelper
from sklearn.metrics import precision_recall_curve

matrix_helper = MatrixHelper()

# Read pickle
matrices = matrix_helper.load_from_parse('data/matrices/matrix_cla_incl_history.pickle')

precision_list = []
recall_list = []
for i in range(10):
    # Instantiate Prediction Helper Class and predict values for compare matrix with an SVM
    prediction_helper = PredictionHelper()
    prediction_helper.calculate_validation_compare_matrix(matrices, sampling_factor=(2.0/3), prediction_type='LinearSVC', crop_matrix=False)
    compare_matrix = prediction_helper.get_compare_matrix()

    # Compute Precision-Recall
    precision, recall, thresholds = precision_recall_curve(np.array(compare_matrix[:, 2], dtype='f'), np.array(compare_matrix[:, 1], dtype='f'))

    precision_list.append(precision[1])
    recall_list.append(recall[1])
    # Print
    print('|{:^11.3f}|{:^8.3f}|{:5.1f}min|'.format(precision[1], recall[1], prediction_helper.time))

precision_average = sum(precision_list) / float(len(precision_list))
recall_average = float(sum(recall_list)) / len(recall_list)
print('Average precision: {}'.format(precision_average))
print('Average recall: {}'.format(recall_average))

|   0.767   | 0.651  |  0.1min|
|   0.761   | 0.621  |  0.1min|
|   0.756   | 0.609  |  0.1min|
|   0.740   | 0.621  |  0.1min|
|   0.774   | 0.614  |  0.1min|
|   0.756   | 0.639  |  0.1min|
|   0.746   | 0.631  |  0.1min|
|   0.732   | 0.596  |  0.1min|
|   0.737   | 0.662  |  0.1min|
|   0.740   | 0.629  |  0.1min|
Average precision: 0.750859168215
Average recall: 0.627121464226


# Das folgende Skript ist für das Testen verschiedener features

In [3]:
import matplotlib.pyplot as plt
import numpy as np

from imports.matrix_helper import MatrixHelper
from imports.prediction_helper import PredictionHelper
from sklearn.metrics import precision_recall_curve

matrix_helper = MatrixHelper()

for filename in ['incl_current']:
    # Read pickle
    matrices = matrix_helper.load_from_parse('data/matrices/matrix_cla_' + filename + '.pickle')

    # Instantiate Prediction Helper Class and predict values for compare matrix with an SVM
    prediction_helper = PredictionHelper()
    prediction_helper.calculate_validation_compare_matrix(matrices, sampling_factor=(2.0/3), prediction_type='LinearSVC', crop_matrix=False)
    compare_matrix = prediction_helper.get_compare_matrix()

    # Compute Precision-Recall
    precision, recall, thresholds = precision_recall_curve(np.array(compare_matrix[:, 2], dtype='f'), np.array(compare_matrix[:, 1], dtype='f'))

    # Print
    vulnerable_testset = matrix_helper.get_vulnerable_percentage(np.array(compare_matrix[:, 2], dtype='f'))
    vulnerable_prediction = matrix_helper.get_vulnerable_percentage(np.array(compare_matrix[:, 1], dtype='f'))
    print('|{:37}|{:^11.3f}|{:^8.3f}|{:16.1f}%|{:19.1f}%|({:5.1f}min)'.format(filename, precision[1], recall[1], vulnerable_testset, vulnerable_prediction, prediction_helper.time))

|incl_current                         |   0.669   | 0.376  |             9.8%|                5.5%|(  0.1min)


## Ergebnisse for includes hisotry matrix

## Ergebnisse mit div Features
| Matrix (Classification)             | Precision | Recall | Vul. in Testset | Vul. in Prediction |
|-------------------------------------|-----------|--------|-----------------|--------------------|
| Includes (current)                  | 0.786     | 0.342  | 9.8%            | 4.3%               |
| Includes (history, with duplicates) | 0.8       | 0.661  | 17.8%           | 14,7%              |
| Includes (history)                  | 0.783     | 0.631  | 9.6%            | 7.7%               |
| Conditions (current)                | 0.794     | 0.097  | 9.8%            | 1.2%               |
| Conditions (history, with duplicates)| 0.88      | 0.517  | 17.4%           | 10.2%              |
| Conditions (history)                | 0.892     | 0.397  | 5.9%            | 2.6%               |
| Names (current)                     | 0.75      | 0.038  | 9.8%            | 0.5%               |
| Names (history, with duplicates)    | 0.671     | 0.483  | 13.5%           | 9.7%               |
| Names (history)                     | 0.556     | 0.126  | 2.1%            | 0.5%               |
| Defs (current)                      |   0.679   | 0.068  |             9.8%|                1.0%|
| Defs (history)                      |   0.594   | 0.264  |             5.0%|                2.2%|

## Ergebnisse mit div Features und LinearSVC
| Matrix (Classification)             | Precision | Recall | Vul. in Testset | Vul. in Prediction |
|-------------------------------------|-----------|--------|-----------------|--------------------|
|incl_current                         |   0.605   | 0.387  |             9.8%|                6.3%|(  0.1min)
|incl_history                         |   0.724   | 0.614  |             9.6%|                8.1%|(  0.1min)
|cond_current                         |   0.786   | 0.165  |             9.8%|                2.1%|(  0.1min)
|cond_history                         |   0.832   | 0.501  |             5.9%|                3.6%|(  0.1min)
|names_current                        |   0.662   | 0.095  |             9.8%|                1.4%|(  0.0min)
|names_history                        |   0.733   | 0.185  |             2.1%|                0.5%|(  0.0min)
|defs_current                         |   0.792   | 0.075  |             9.8%|                0.9%|(  0.5min)
|defs_history                         |   0.620   | 0.381  |             5.0%|                3.1%|(  0.5min)
|calls_current                        |   0.734   | 0.401  |             9.8%|                5.4%|(  1.8min)
|calls_history                        |   0.774   | 0.704  |            14.0%|               12.7%|(  4.6min)